In [ ]:
plist_path = "/private/var/mobile/Containers/Data/Application/8C15CF35-580B-488E-9B29-D89BDB144E80/Library/Application Support/RecentFiles.plist"
file_path="/private/var/mobile/Containers/Data/Application/77881549-3FA6-4E4B-803F-D53B172FC865/Documents/Notebooks/PythonistalabTest.ipynb"

In [ ]:
import re
import plistlib

# Allowed file extensions
allowed_extensions = ['txt','doc','pdf','ipynb', 'py']
extension_pattern = '|'.join(allowed_extensions)
# Simpler Unicode regex for file paths
pattern = rf'(?:(?:/)?[\w\-\.]+(?:/[\w\-\.]+)*)\.(?:{extension_pattern})'

def extract_paths(binary_data):
    """Extract file paths from a binary string using Unicode regex."""
    # Decode with 'replace' to handle invalid sequences
    text_data = binary_data.decode('utf-8', errors='replace')
    try:
        matches = re.findall(pattern, text_data)
        return matches
    except re.error as e:
        print(f"Regex error: {e}")
        return []

def get_current_notebook_path():
    """Get the most recent notebook path from RecentFiles.plist."""
    plist_path = "/private/var/mobile/Containers/Data/Application/8C15CF35-580B-488E-9B29-D89BDB144E80/Library/Application Support/RecentFiles.plist"
    with open(plist_path, 'rb') as f:
        return extract_paths(plistlib.load(f)[0])[-1]
    

# Test the functions
if __name__ == "__main__":
    print(f"{get_current_notebook_path() = }\n")
    
    # Example usage
    print("Example for 'extract_paths':")
    binary_data = b"data /Users/test/file.txt\xA4invalid/home/\xe6\x96\x87\xe6\xa1\xa3/doc.pdf PythonistalabTest.ipynb"
    print(extract_paths(binary_data))
    
    # Full plist contents
    print("\nFile paths in RecentFiles.plist:")
    plist_path = "/private/var/mobile/Containers/Data/Application/8C15CF35-580B-488E-9B29-D89BDB144E80/Library/Application Support/RecentFiles.plist"
    with open(plist_path, 'rb') as f:
        # plist_data is a list of binary strings (not dictionaries)
        for i,binary_string in enumerate(plistlib.load(f)):
            print(i,extract_paths(binary_string),end='\n\n')

In [ ]:
import timeit

setup = '''
matches = [b'test' * 1000 for _ in range(1000)]
def with_comprehension():
    return [m_d for match in matches for m_d in (match.decode('utf-8', errors='ignore'),) if m_d]

def with_loop():
    result = []
    for match in matches:
        m_d = match.decode('utf-8', errors='ignore')
        if m_d:
            result.append(m_d)
    return result

def with_generator():
    return list(m_d for match in matches if (m_d := match.decode('utf-8', errors='ignore')))
'''

print("List comprehension:", timeit.timeit('with_comprehension()', setup=setup, number=1000))
print("For loop with append:", timeit.timeit('with_loop()', setup=setup, number=1000))
print("List(generator):", timeit.timeit('with_generator()', setup=setup, number=1000))

For loop with append: 0.6737396669923328


List(generator): 0.7003315419424325


In [ ]:
 matches

In [ ]:
import os
os.environ

In [ ]:
for i in range(len(plist_data)):
  print(f'({plist_data[i][:4] = }\n {plist_data[i][4]+plist_data[i][5]*256 =}\n {len(plist_data[i])=}\n{','.join(str(c) for c in plist_data[i][6:40])=} ) ')

In [ ]:
plist_data[0][40:80]


In [ ]:
import plistlib

# Path to the .plist file
#plist_path = "/private/var/mobile/Containers/Data/Application/8C15CF35-580B-48E9-9F5A-7B9D5A2E0940/Library/Application Support/your_plist_file.plist"

try:
    # Read the .plist file using plistlib
    with open(plist_path, 'rb') as plist_file:
        plist_data = plistlib.load(plist_file)
    
    if not plist_data:
        print("No recent files found.")
        exit()
    
    # Parse the first binary string (most recent file)
    binary_string = plist_data[0]['book']
    
    # Validate the header
    if binary_string[:4] != b'book':
        print("Unexpected format: First 4 bytes are not 'book'.")
        exit()
    
    # Extract the length (bytes 4-5, little-endian)
    record_length = int.from_bytes(binary_string[4:6], 'little')
    print(f"Record length: {record_length} bytes")
    
    # Parse the header
    header_size = 38
    if len(binary_string) < header_size:
        print("Binary string too short to contain a header.")
        exit()
    
    # Extract the first offset (bytes 11-18, little-endian)
    next_offset = int.from_bytes(binary_string[11:19], 'little')
    print(f"First offset from header: {next_offset}")
    
    # Parse the fields
    pos = next_offset
    fields = []
    while pos < len(binary_string):
        # Check for the type indicator (e.g., 0,0,0,4,16)
        if pos + 5 > len(binary_string):
            break
        type_indicator = binary_string[pos:pos+5]
        pos += 5
        
        # Extract the offset to the next field (8 bytes, little-endian)
        if pos + 8 > len(binary_string):
            break
        next_field_offset = int.from_bytes(binary_string[pos:pos+8], 'little')
        pos += 8
        
        # Extract the field data (from current position to next_field_offset)
        if next_field_offset < pos or next_field_offset > len(binary_string):
            break
        field_data = binary_string[pos:next_field_offset]
        
        # Try to decode the field as ASCII if possible
        try:
            field_str = field_data.decode('ascii')
            print(f"Field at offset {pos}: {field_str}")
            fields.append((field_str, pos))
        except UnicodeDecodeError:
            print(f"Field at offset {pos}: (binary data, length {len(field_data)} bytes)")
            fields.append((field_data, pos))
        
        pos = next_field_offset
    
    # Look for the standalone filename and metadata list
    filename = None
    metadata = None
    full_path = None
    for field, offset in fields:
        if isinstance(field, str):
            if field.endswith(('.ipynb', '.py')) and ';' not in field:
                filename = field
                print(f"Standalone filename at offset {offset}: {filename}")
            elif field.endswith(('.ipynb', '.py')) and ';' in field:
                metadata = field
                fields_split = metadata.split(';')
                if fields_split and fields_split[-1].endswith(('.ipynb', '.py')):
                    full_path = fields_split[-1]
                    print(f"Metadata list at offset {offset}: {metadata}")
                    print(f"Full path: {full_path}")

except Exception as e:
    print(f"Failed to deserialize .plist: {e}")

In [ ]:
for i,a in enumerate(plist_data[1]):
  if i%64==0: print()
  c=chr(a)
  print(c if c.isascii() and a>=32 else '.',end='',sep='')
  
print()


In [ ]:
fn=[None]*2
fn[0]='eccf58258e501d971934c668b61ff90fa17b7f29a9a81abad12096dd00790744;00;00000000;00000000;00000000;0000000000000020;com.apple.app-sandbox.read-write;01;01000004;0000000006482a8c;12;/private/var/mobile/Containers/Data/Application/77881549-3FA6-4E4B-803F-D53B172FC865/Documents/Notebooks/PythonistalabTest.ipynb'
fn[1]='afa7edfad4d2ca243dc112b100235b6df391c5342db82a19a696bc783a2f7178;00;00000000;00000000;00000000;0000000000000020;com.apple.app-sandbox.read-write;01;01000004;00000000064800be;12;/private/var/mobile/Containers/Data/Application/77881549-3FA6-4E4B-803F-D53B172FC865/Documents/Notebooks/get_path.py'
[len(f) for f in fn]

In [ ]:
pos=plist_data[0].find(bytes((305-256+1,1))) 
plist_data[0][pos-10:pos+10] 

In [ ]:
pos=plist_data[1].find(bytes((293-256+1,1))) 
plist_data[1][pos-10:pos+10] 

In [ ]:
pos=plist_data[1].find(b'private')
print(pos)
plist_data[1][pos-8:pos+15] 

In [ ]:
import sys
sys.path.insert(0,'/private/var/mobile/Containers/Data/Application/8C15CF35-580B-488E-9B29-D89BDB144E80/Documents/Documents/site-packages')

In [ ]:
import jupytext


In [ ]:
os.chdir(os.path.dirname(get_current_notebook_path()))

In [ ]:
jupytext.write(jupytext.read('PythonistalabTest.ipynb'),'PythonistalabTest.md')


In [ ]:
os.listdir()
 